In [12]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import joblib
from autoIG.epics import BITCOIN_EPIC
from trading_ig.config import config
from trading_ig.rest import IGService
ig_service = IGService(config.username, config.password, config.api_key)
ig = ig_service.create_session()
# Persist the data, so we can train on the same dataset each time. 
# This should be tracked with DVC
LOAD_DATA = False
SAVE_MODEL = False
if LOAD_DATA:
    base_model_results = ig_service.fetch_historical_prices_by_epic(BITCOIN_EPIC,resolution ='1Min',numpoints = 10 )
    base_model_data = base_model_results['prices']
    base_model_data.to_pickle('base_model_data.pkl')
else:
    base_model_data = pd.read_pickle('base_model_data.pkl')

In [13]:
# create the return we would get if we bought. 

In [14]:
# we buy at bid,sell at next 
base_model_data.columns = base_model_data.columns.get_level_values(0)+"_"+base_model_data.columns.get_level_values(1)

In [15]:
base_model_data = base_model_data[['bid_Open','ask_Open']]
base_model_data['ask_Open_s'] = base_model_data['ask_Open'].shift(-1)
base_model_data['r'] = base_model_data['ask_Open_s'] / base_model_data['bid_Open']
base_model_data = base_model_data.dropna()
base_model_data.head(2)

,bid_Open,ask_Open,ask_Open_s,r
DateTime,,,,
2022-12-06 18:56:00,16927.3,17017.3,17034.7,1.006345
2022-12-06 18:57:00,16944.7,17034.7,17027.2,1.004869


In [16]:
# Predit the return from the bid price
X = base_model_data[['bid_Open']]
y = base_model_data['r']
model = LinearRegression()
model.fit(X,y)

LinearRegression()

In [17]:
model.predict(X)

array([1.0058031 , 1.00462497, 1.00513279, 1.00516664, 1.00568122,
       1.00542393, 1.0054781 , 1.00541716, 1.00552549])

In [18]:
# persist the model
if SAVE_MODEL:
    joblib.dump(model,'base_model.pkl')
else:
    model = joblib.load('base_model.pkl')

In [19]:
model.predict(X)

array([1.0058031 , 1.00462497, 1.00513279, 1.00516664, 1.00568122,
       1.00542393, 1.0054781 , 1.00541716, 1.00552549])